<a href="https://colab.research.google.com/github/HosseinSadeghi-dev/Persian_Recognition/blob/main/sync_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install telebot
!pip install pyTelegramBotAPI

In [4]:
import cv2
import telebot
import numpy as np
from keras.models import load_model

model = load_model("/content/drive/MyDrive/model_Persian_recognition.h5")
bot = telebot.TeleBot("2003265436:AAEBC8nFhLDY3iZ0RgW2DPcXY3n9buTg4jw")

scale = 225


@bot.message_handler(commands=['start'])
def say_hello(messages):
    bot.send_message(messages.chat.id, f'Welcome {messages.from_user.first_name}')
    bot.send_message(messages.chat.id, f'Here you can Recognition Persians 🇮🇷 from who are not Persian')
    bot.send_message(messages.chat.id, f'Send Portrait From Person You Want to Know Where he/she is From')


@bot.message_handler(content_types=['photo'])
def photo(message):
    src = bot.get_file(message.photo[-1].file_id).file_path
    downloaded_file = bot.download_file(src)
    with open("/content/drive/MyDrive/test-telegram-bots/" + src, 'wb') as new_file:
        new_file.write(downloaded_file)

    image = cv2.imread("/content/drive/MyDrive/test-telegram-bots/" + src)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (scale, scale))
    image = image / scale
    image = image.reshape(1, scale, scale, 3)
    pred = model.predict([image])

    res = np.argmax(pred)
    if res == 0:
        bot.reply_to(message, 'Persian 🇮🇷')
    else:
        bot.reply_to(message, 'Not Persian')


bot.polling()
